# Analysis and Debugging Notebook

This notebook breaks down the data processing and calculation steps from `search_data.py` to allow for step-by-step analysis of a single simulation run. This is useful for debugging why certain values might be reported as `N/A`.

## 1. Setup and Imports

First, we import the necessary functions and modules. We add the current directory to `sys.path` to ensure that local modules like `run_evaluation` and `load_input_yaml` are found.

In [1]:
import os
import sys
import pprint

# Add the current directory to the path to find our custom modules
if '.' not in sys.path:
    sys.path.insert(0, '.')

from load_input_yaml import load_params
import run_evaluation

print("Setup complete. Modules imported.")

Setup complete. Modules imported.


## 2. Configuration

Specify the path to the simulation run you want to analyze. Change the `run_path` variable to point to any run directory containing an `input.yaml` file.

I've pre-filled it with one of the paths from your example that produced `N/A` values.

In [ ]:
# IMPORTANT: Set this path to the specific run you want to debug
run_path = "../data/20251221/45"  # Example path with N/A and nan values
# run_path = "../data/20251216/34" # Example path with mostly valid values

print(f"Analyzing run: {os.path.abspath(run_path)}")

Analyzing run: /home/piet/Masterarbeit/data/20251221/45


## 3. Load Initial Parameters from `input.yaml`

This step reads the `input.yaml` file for the selected run. It's equivalent to what `find_matching_runs` does. This helps verify that the basic run parameters are being loaded correctly.

In [3]:
try:
    metro_params, gauge_params = load_params(os.path.join(run_path, 'input.yaml'))
    print("--- Metropolis Parameters ---")
    pprint.pprint(vars(metro_params))
    print("--- Gauge Observable Parameters ---")
    pprint.pprint(vars(gauge_params))
except FileNotFoundError as e:
    print(f"ERROR: Could not find input.yaml in the specified path: {e}")
except Exception as e:
    print(f"An error occurred while loading parameters: {e}")

--- Metropolis Parameters ---
{'L0': 46,
 'L1': 46,
 'L2': 46,
 'L3': 46,
 'Nc': 2,
 'Nd': 4,
 'Ndims': 4,
 'beta': 2.2,
 'delta': 0.1,
 'epsilon1': 0.0,
 'epsilon2': 0.0,
 'nHits': 10,
 'nSweep': 200000,
 'seed': 30170}
--- Gauge Observable Parameters ---
{'RetraceU_filename': 'Retrace.out',
 'W_Lmu_Lnu_pairs': [(2, 2), (3, 3)],
 'W_mu_nu_filename': 'W_mu_nu.out',
 'W_mu_nu_pairs': [(0, 1)],
 'W_temp_L_T_pairs': [(1, 1),
                      (1, 2),
                      (1, 3),
                      (1, 4),
                      (1, 5),
                      (1, 6),
                      (1, 7),
                      (1, 8),
                      (1, 9),
                      (1, 10),
                      (1, 11),
                      (1, 12),
                      (2, 1),
                      (2, 2),
                      (2, 3),
                      (2, 4),
                      (2, 5),
                      (2, 6),
                      (2, 7),
                      (2, 8),
 

## 4. Perform Core Calculation

This is the most critical step. We call `run_evaluation.get_or_calculate(run_path)`, which is responsible for all the heavy lifting (calculating `r0`, `chi`, etc.).

We will print the entire dictionary that this function returns. **Look closely at this output.** If a key like `'r0_chi'` is missing or its value is `None` or `NaN`, it explains the `N/A` in the script's output.

In [6]:
print("Running evaluation... This may take a moment.")
calculated_data = run_evaluation.get_or_calculate(run_path)

print("--- Result of get_or_calculate ---")
pprint.pprint(calculated_data)

Running evaluation... This may take a moment.
--- Result of get_or_calculate ---
{'F_chi': {'1.5': 0.35569921764579354,
           '2.5': 0.2912079246519986,
           '3.5': 0.2694542184061974,
           '4.5': 0.2544074637279381,
           '5.5': 0.2892753318375101,
           '6.5': 0.2818285232641068,
           '7.5': -0.648077593192862},
 'V_R': {'1': 0.5050150470413277,
         '2': 0.89840221608955,
         '3': 1.2669009114773677,
         '4': 1.6288427148106102,
         '5': 1.990901114449961,
         '6': 2.34851395685087,
         '7': 2.7053592485024964,
         '8': 3.0615158490534586},
 'W_R_T': {'1,1': 0.5692378520802006,
           '1,10': 0.006325990820050125,
           '1,11': 0.00384550048245614,
           '1,12': 0.0023375723067669174,
           '1,2': 0.34120484674185464,
           '1,3': 0.20674885679197996,
           '1,4': 0.12556713245614037,
           '1,5': 0.07630120457142857,
           '1,6': 0.046369950446115284,
           '1,7': 0.028180

## 5. Extract and Display Final Values

Finally, we simulate the logic from `process_row` in `search_data.py`. We attempt to extract the desired fields from the `calculated_data` dictionary and print them one by one. This makes it clear which fields were successfully calculated and which were not.

In [5]:
output_fields = ["r0", "r0_err", "r0_chi", "r0_chi_err", "W_R_T", "V_R"]

print("--- Extracted Values ---")
if calculated_data and 'error' not in calculated_data:
    for field in output_fields:
        value = calculated_data.get(field, "N/A (Not Found)")
        print(f"{field+':':<12} {value}")
else:
    print("Could not extract values because the calculation step produced an error or no data.")

--- Extracted Values ---
r0:          3.7282703764614493
r0_err:      0.0
r0_chi:      None
r0_chi_err:  None
W_R_T:       {'1,1': 0.5692378520802006, '1,2': 0.34120484674185464, '1,3': 0.20674885679197996, '1,4': 0.12556713245614037, '1,5': 0.07630120457142857, '1,6': 0.046369950446115284, '1,7': 0.028180425145363404, '1,8': 0.017127148177944862, '1,9': 0.01040907466491228, '1,10': 0.006325990820050125, '1,11': 0.00384550048245614, '1,12': 0.0023375723067669174, '2,1': 0.34120386020050125, '2,2': 0.13634418468671677, '2,3': 0.05705516774686716, '2,4': 0.02419766695238095, '2,5': 0.010302691727819549, '2,6': 0.004391004140601504, '2,7': 0.0018724822556390977, '2,8': 0.0007973220821553884, '2,9': 0.0003402063347820552, '2,10': 0.00014518172722673432, '2,11': 6.178522253337594e-05, '2,12': 2.592676713039599e-05, '3,1': 0.2067491014536341, '3,2': 0.05705574197243107, '3,3': 0.017378371556390975, '3,4': 0.005469312823057645, '3,5': 0.001740363552631579, '3,6': 0.000555179964708371, '3,7': 